In [1]:
import os
import shutil
# import pydicom
import pandas as pd
import SimpleITK as sitk
import re



def DicomHeadInforExtraction(DicomFilePath,InformationNeed,OutputRoot=None,SerPredictionModel=None,SerPreResultExplainDict=None):
    ResultDict={}
    NeedCheck=[]
    n=0
    for root,dirs,files in os.walk(DicomFilePath,topdown=False):
        if files!=[]:
#             for f in files:
#                 if 'dcm' not in f:
#                     FilePathRaw=os.path.join(root,f)
#                     FilePathUse=os.path.join(root,f+'.dcm')
#                     shutil.move(FilePathRaw,FilePathUse)
            series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(root)
#             print series_IDs
            if series_IDs!=None:
                for s in series_IDs:
#                     print s
                    series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(root, s)
#                     print series_file_names
                    ImageFile=series_file_names[0]
#                     PatientID=ImageFile.split('/')[6]
#                     SubfolderName=ImageFile.split('/')[7].upper()
                    ResultDict[n]={'DicomRawPaths':series_file_names,'NumberOfFile':len(series_file_names)}
                    image_reader = sitk.ImageFileReader()
                    image_reader.LoadPrivateTagsOn()
                    image_reader.SetFileName(ImageFile)
                    try:
                        img = image_reader.Execute()
                    except Exception as e:
                        NeedCheck.append((PatientID,series_file_names,e))
                        continue
                    h=0
                    for tag,name in InformationNeed:
                        try:
                            value=img.GetMetaData(tag)
                        except:
#                             print tag,name
                            value=None
                            h+=1
                        ResultDict[n].update({name:value})
                    if h>=9:
                        ResultDict.pop(n)
                        NeedCheck.append((PatientID,series_file_names,'No enough head inform'))
                        continue
                    if OutputRoot!=None:
                        series_reader = sitk.ImageSeriesReader()
                        series_reader.SetFileNames(series_file_names)
                        NewFilePath=os.path.join(OutputRoot,PatientID,SubfolderName)
                        try:
                            os.makedirs(NewFilePath)
                        except:
                            pass
                        try:
                            image3D = series_reader.Execute()
                        except Exception as e:
                            NeedCheck.append((PatientID,series_file_names,e))
                            continue
                        SerDes=ResultDict[n]['SeriesDescription_0008_103e']
                        SerP=''
                        if SerPredictionModel!=None:
                            try:
                                TE=float(ResultDict[n]['EchoTime_0018_0081'])
                                TR=float(ResultDict[n]['RepetitionTime_0018_0080'])
                            except:
                                print 'TE or TR not exist'
                                pass
                            try:
                                SerPRaw=SerPredictionModel.predict([[TE,TR]])[0]
                                SerP=SerPreResultExplainDict[SerPRaw]
                                ResultDict[n].update({'SerPredict':SerP})
                            except:
                                print 'SerPredictionModel not usable | SerPreResultExplainDict not suitable, please check'
                        if SerDes!=None and any(re.search(i,SerDes,re.I)!=None for i in ['T1','T2','Flair','c']):
                            SerDesUse='_'.join([SerP,str(n)]+re.findall('\w+',SerDes)).upper()
                            sitk.WriteImage(image3D,NewFilePath+'/'+SerDesUse+'.nii.gz')
                            
                        else:
                            SerDesUse=SerP+'_'+str(n)
                            sitk.WriteImage(image3D,NewFilePath+'/'+SerDesUse+'.nii.gz')
                        ResultDict[n].update({'NiiFilePath':NewFilePath+'/'+SerDesUse+'.nii.gz'})
                    else:
                        pass
                    n+=1
                    
    ResultDf=pd.DataFrame(ResultDict).T
    return ResultDf,NeedCheck

In [2]:
import os
import shutil
# import pydicom
import pandas as pd
import SimpleITK as sitk
import re
InformationNeed=[["0008|0020",'StudyDate_0008_0020'],
                 ["0008|0050",'AccessionNumber_0008_0050'],
                 ["0008|0070",'Manufacturer_0008_0070'],
                 ["0008|103e",'SeriesDescription_0008_103e'],
                 ["0018|0023",'MRAcquisitionType_0018_0023'],
                 ["0018|0025",'AngioFlag_0018_0025'],["0018|1314",'FlipAngle_0018_1314'],
                 ["0018|0050",'SliceThickness_0018_0050'],
                 ["0018|0080",'RepetitionTime_0018_0080'],
                 ["0018|0081",'EchoTime_0018_0081'],
                 ["0018|0082",'InversionTime_0018_0082'],
                 ["0018|0084",'ImagingFrequency_0018_0084'],
                 ["0018|0087",'MagneticFieldStrength_0018_0087'],
                 ["0020|0011",'SeriesNumber_0020_0011'],
                 ["0010|0030",'DOB_0010_0030'],
                 ["0010|0040",'Gender_0010_0040'],['0010|0020','PatientID_0010_0040'],['0028|0010','Rows_0028_0010'],
                 ['0028|0011','Columns_0028_0011'],['0028|0030','PixelSpacing_0028_0030']]
# Use='D:/SH_MH_W/260-293/2/100482110/T2/PN100482110_20180612_4_17_01.dcm'
# sitk.ImageSeriesReader.GetGDCMSeriesIDs('D:/SH_MH_W/260-293/2/100482110/+C/')

In [3]:

import SimpleITK as sitk

def ReadHeadInform(img,InformationNeed):
    ResultDict={}
    NeedCheck=False
    h=0
    for tag,name in InformationNeed:
        try:
            value=img.GetMetaData(tag)
        except:
    #                             print tag,name
            value=None
            h+=1
        ResultDict.update({name:value})
    if h>=9:
        NeedCheck=True
    return NeedCheck,ResultDict

In [4]:
def ReadImg(root,filename):
    image_reader = sitk.ImageFileReader()
    image_reader.LoadPrivateTagsOn()
    image_reader.SetFileName(os.path.join(root,filename))
    temp=image_reader.Execute()
    return temp

In [5]:
import re
def SerNameUse(SerName,SerNameNeedList=['DWI','T1','T2']):
    SerListUse=sorted(SerNameNeedList,reverse=False)
    for i in SerListUse:
        if re.search(i,SerName,re.I)!=None:
            return i

In [6]:
import re
import os
def Regular_Image2Ser(root,FinialPathRoot,InformationNeed):
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(root)
#             print series_IDs
    NeedCheck=[]
    FinialResult=[]
    n=0
    for s in series_IDs:
#                     print s
        series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(root, s)
#                     print series_file_names
        img=ReadImg(root,series_file_names[0])
        Checking,ResultDict=ReadHeadInform(img,InformationNeed)
        SerDes=str(ResultDict['SeriesDescription_0008_103e'])
#             print SerDes
        SerDesUse=SerNameUse(SerName=SerDes)
        if SerDesUse=='T2':
            if Checking:
                NeedCheck.append((root,series_file_names))
            else:
                series_reader = sitk.ImageSeriesReader()
                series_reader.SetFileNames(series_file_names)
                try:
                    Image = series_reader.Execute()
                except:
                    NeedCheck.append((root,series_file_names))
                    continue

                Path=os.path.join(FinialPathRoot,SerDesUse+'.nii.gz')
                ResultDict.update({'NumberOfSlice':len(series_file_names),'NiiPath':Path,'DicomPath':root})
                if os.path.isfile(Path):
                    pass
                else:
                    sitk.WriteImage(Image,Path,True)
                FinialResult.append(ResultDict)
                
    if  FinialResult!=[]:   
        return  NeedCheck,FinialResult    
    else:
        raise ValueError

In [80]:
float(int(1.0))

1.0

In [7]:
import numpy as np
import SimpleITK as sitk
def Image2Ser(root,FilesUse,Subgroup,NSer,FinialPathRoot,InformationNeed):
    NeedCheck=[]
    FinialPath=[]
#     print(FilesUse)
    temp=ReadImg(root,FilesUse[0])
    Checking,ResultDict=ReadHeadInform(temp,InformationNeed)
    
    if Checking:
        NeedCheck.append({'Path':root,'Files':FilesUse,'NubFiles':len(FilesUse)})
    else:
        SerDes=str(ResultDict['SeriesDescription_0008_103e'])
#             print SerDes
        SerDesUse=SerNameUse(SerName=SerDes)
        TempArray=sitk.GetArrayFromImage(temp)
        _,y,x= TempArray.shape
        dtype= TempArray.dtype
        if SerDesUse=='T1':
            zz=len(FilesUse)/40.0
            if zz!=float(int(zz)):
                z1=len(FilesUse)/NSer[SerDesUse]
                if z1!=float(int(z1)):
                    z=None
                    NeedCheck.append({'Path':root,'Files':FilesUse,'NubFiles':len(FilesUse)})
                    pass
                else:
                    z=int(z1)
                    ZZ=NSer[SerDesUse]+1
            else:
                z=40
                ZZ=zz+1
        else:
            z1=len(FilesUse)/NSer[SerDesUse]
            
            if z1!=float(int(z1)):
                z=None
                NeedCheck.append({'Path':root,'Files':FilesUse,'NubFiles':len(FilesUse)})
                pass 
            else:
                z=int(z1)
                ZZ=NSer[SerDesUse]+1
        if z!=None:
            ResultDict.update({'NumberOfSlice':z,'NubFiles':len(FilesUse)})
            n=1
    
            while n<ZZ:
                
                ArrayNeed=np.zeros([z,y,x])
                k=0
                FileGroup=FilesUse[:z]
                series_reader = sitk.ImageSeriesReader()
                series_reader.SetFileNames([os.path.join(root,i) for i in FileGroup])
#                 try:
                Image = series_reader.Execute()
#                 except:
#                     NeedCheck.append({'Path':root,'Files':FilesUse,'NubFiles':len(FilesUse)})
#                     pass
                for f in FileGroup:
#                     temp=ReadImg(root,f)
#                     ArrayNeed[k,:,:]=sitk.GetArrayFromImage(temp)
                    FilesUse.remove(f)
                    k+=1
#                 Image=sitk.GetImageFromArray(ArrayNeed.astype(dtype))
                filename='_'.join([SerDesUse,Subgroup,'%d.nii.gz' % n])
                Path=os.path.join(FinialPathRoot,filename)
                
                FinialPath.append(Path)
                if os.path.isfile(Path):
                    ImageRaw=sitk.ReadImage(Path)
                    ImageRaw.CopyInformation(Image)
                    sitk.WriteImage(ImageRaw,Path,True)
                    pass
                else:
                    sitk.WriteImage(Image,Path,True)
                
                
                n+=1
            ResultDict.update({'NiiPath':FinialPath,'DicomPath':root,'TotalNumberDicom':len(FilesUse)})
    return NeedCheck,ResultDict

In [8]:
import re
def Special_Image2Ser(root,FinialPathRoot,InformationNeed,NSer={'DWI':2.0,'T1':7.0,'T2':1.0}):
    NeedCheck,FinialResult=[],[]
    for r,folder,files in os.walk(root):
        if files!=[]:
            FilesUse_pre=list(filter(lambda x: re.search('_\d{8}(_MR)?_\d+_\d+_\d+\.dcm',x)!=None,files))
            SubSer=set(map(lambda x:'_'.join( x.split('_')[:-1]),FilesUse_pre))
#             print(SubSer)
            FilesUseList=[]
            for i in SubSer:
#                 print(i)
                R=list(filter(lambda x: i in x,FilesUse_pre))
                FilesUseList.append((i,sorted(R,key=lambda x: int(x.split('_')[-1].split('.')[0]))))
            for S,FilesUse in FilesUseList:
                S_use='_'.join(S.split('_')[-2:])
                N,Fr=Image2Ser(root=r,FilesUse=FilesUse,Subgroup=S_use,NSer=NSer,FinialPathRoot=FinialPathRoot,InformationNeed=InformationNeed)
                NeedCheck.extend(N)
                FinialResult.append(Fr)
    return NeedCheck,FinialResult

In [63]:
RootNeed[0]

'/home2/nye/SH_MH_W/RawDicom/101915985/T2'

In [10]:
RootNeed=[]
for root,folder,files in os.walk('/home2/nye/SH_MH_W/RawDicom'):
    if files!=[] and any('dcm' in i for i in files):
        RootNeed.append(root)

In [53]:
len(RootNeed)

872

In [27]:
FinialPathRoot='/home2/nye/SH_MH_W/NiiUse'
# CantRun=[]
# import random

# for root,folder,files in os.walk('/home2/nye/SH_MH_W/RawDicom/thy1/'):
#     if files!=[] and any('dcm' in i for i in files):
        
def tempfun(root):
    N=[]
    Fr=[]
    CantRun={}
    FinialPathRootUse=os.path.join(FinialPathRoot,root.split('/')[-2])
    try:
        os.makedirs(FinialPathRootUse)
    except:
        pass
#         print  FinialPathRootUse
    try:
        N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                                  root=root,
                                  InformationNeed=InformationNeed)
    except:
        Files=os.listdir(root)
        CantRun={'Path':root,'Files':Files,'FileNub':len(list(filter(lambda x: 'dcm' in x,Files)))}

    return N,Fr,CantRun

In [28]:
from subprocess import call
from multiprocessing import Pool
a=list(Pool(processes=30).imap(tempfun,RootNeed))

In [14]:
for a

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [29]:
CantRun=[]
NeedCheck=[]
Result=[]
for x,y,z in a:
    if z!={}:
        CantRun.append(z)
    NeedCheck.extend(x)
    Result.extend(y)

In [33]:
CantRun

[{'FileNub': 78,
  'Files': ['PN101429930_20180301_5_1_09.dcm',
   'PN101429930_20180301_5_1_026.dcm',
   'PN102456166_20180321_5_1_026.dcm',
   'PN102456166_20180321_5_1_033.dcm',
   'PN102456166_20180321_5_1_08.dcm',
   'PN102456166_20180321_5_1_03.dcm',
   'PN101429930_20180301_5_1_012.dcm',
   'PN102456166_20180321_5_1_017.dcm',
   'PN102456166_20180321_5_1_027.dcm',
   'PN101429930_20180301_5_1_023.dcm',
   'PN101429930_20180301_5_1_01.dcm',
   'PN101429930_20180301_5_1_027.dcm',
   'PN102456166_20180321_5_1_04.dcm',
   'PN102456166_20180321_5_1_036.dcm',
   'PN102456166_20180321_5_1_021.dcm',
   'PN101429930_20180301_5_1_035.dcm',
   'PN102456166_20180321_5_1_019.dcm',
   'PN102456166_20180321_5_1_041.dcm',
   'PN101429930_20180301_5_1_06.dcm',
   'PN102456166_20180321_5_1_042.dcm',
   'PN102456166_20180321_5_1_025.dcm',
   'PN101429930_20180301_5_1_016.dcm',
   'PN102456166_20180321_5_1_035.dcm',
   'PN102456166_20180321_5_1_018.dcm',
   'PN101429930_20180301_5_1_036.dcm',
   'P

In [34]:
import pandas as pd
HeadInform=pd.DataFrame({n:i for n,i in enumerate(Result)}).T

In [45]:
HeadInform.head()

,AccessionNumber_0008_0050,AngioFlag_0018_0025,Columns_0028_0011,DOB_0010_0030,DicomPath,EchoTime_0018_0081,FlipAngle_0018_1314,Gender_0010_0040,ImagingFrequency_0018_0084,InversionTime_0018_0082,...,NiiPath,NumberOfSlice,PatientID_0010_0040,PixelSpacing_0028_0030,RepetitionTime_0018_0080,Rows_0028_0010,SeriesDescription_0008_103e,SeriesNumber_0020_0011,SliceThickness_0018_0050,StudyDate_0008_0020
0,AN18012901560029,N,512,19670125,/home2/nye/SH_MH_W/RawDicom/thy5/102446022/T2,82.88,90,F,63.849391,0,...,[/home2/nye/SH_MH_W/NiiUse/102446022/T2_4_1_1....,18,PN102446022,0.2734\0.2734,4520,512,OAx T2 fs,4,4,20180205
1,AN18012901560029,N,256,19670125,/home2/nye/SH_MH_W/RawDicom/thy5/102446022/DWI,88.5,90,F,63.849388,0,...,[/home2/nye/SH_MH_W/NiiUse/102446022/DWI_5_1_1...,22,PN102446022,0.5469\0.5469,6200,256,OAx DWI b=800,5,4,20180205
2,AN18012901560029,N,512,19670125,/home2/nye/SH_MH_W/RawDicom/thy5/102446022/DCE,1.872,12,F,63.849388,0,...,[/home2/nye/SH_MH_W/NiiUse/102446022/T1_6_1_1....,45,PN102446022,0.2734\0.2734,5.872,512,AX T1 FSPGR +c 1.75Ph,6,4,20180205
3,AN18040106650052,N,512,19521216,/home2/nye/SH_MH_W/RawDicom/thy5/102495333/T2,82.432,90,F,63.849291,0,...,[/home2/nye/SH_MH_W/NiiUse/102495333/T2_5_1_1....,18,PN102495333,0.2734\0.2734,4480,512,OAx T2 fs,5,4,20180402
4,AN18040106650052,N,256,19521216,/home2/nye/SH_MH_W/RawDicom/thy5/102495333/DWI,85.6,90,F,63.849306,0,...,[/home2/nye/SH_MH_W/NiiUse/102495333/DWI_6_1_1...,18,PN102495333,0.5469\0.5469,6200,256,OAx DWI b=800,6,4,20180402


In [37]:
HeadInform.to_excel('./HeadInform_20181010.xlsx')

In [36]:
NeedCheckDf=pd.DataFrame({n:i for n,i in enumerate(NeedCheck)}).T

In [38]:
NeedCheckDf.to_excel('./NeedCheckDf_20181010.xlsx')

In [137]:
MoreThan7=[]
for d,i in zip(HeadInform.NiiPath,HeadInform.DicomPath):
    try:
        if len(d)>7:
            MoreThan7.append(i)
    except:
        pass

In [40]:
len(NeedCheckDf)

17

In [139]:
HeadInform[HeadInform.NiiPath.isnull()]

,AccessionNumber_0008_0050,AngioFlag_0018_0025,Columns_0028_0011,DOB_0010_0030,DicomPath,EchoTime_0018_0081,FlipAngle_0018_1314,Gender_0010_0040,ImagingFrequency_0018_0084,InversionTime_0018_0082,...,NumberOfSlice,PatientID_0010_0040,PixelSpacing_0028_0030,RepetitionTime_0018_0080,Rows_0028_0010,SeriesDescription_0008_103e,SeriesNumber_0020_0011,SliceThickness_0018_0050,StudyDate_0008_0020,TotalNumberDicom
193,AN16052701560525,N,512,19600117,NaN,1.636,12,F,63.849673,0,...,NaN,PN101914505,0.4883\0.4883,5.54,512,AX T1 FSPGR +c 1.75Ph,7,4,20160527,NaN
218,AN18070906650165,N,512,19660211,NaN,1.872,12,F,63.848941,0,...,NaN,PN102585421,0.2734\0.2734,5.872,512,AX T1 FSPGR +c 1.75Ph,7,4,20180719,NaN
273,AN18080606650145,N,512,19670323,NaN,1.872,12,F,63.849402,0,...,NaN,PN102611664,0.2734\0.2734,5.872,512,AX T1 FSPGR +c 1.75Ph,6,4,20180806,NaN
289,AN16101501560113,N,512,19810216,NaN,10.328,90,M,63.849475,0,...,NaN,PN102031635,0.4883\0.4883,480,512,OAx T1,5,4,20161020,NaN
303,AN16030203980014,N,512,19940620,NaN,1.636,12,M,63.849601,0,...,NaN,PN101829633,0.4883\0.4883,5.54,512,AX T1 FSPGR +c,7,4,20160303,NaN
315,AN18042306680300,N,512,19680910,NaN,1.872,12,M,63.849484,0,...,NaN,PN102514321,0.2734\0.2734,5.868,512,AX T1 FSPGR +c 1.75Ph,7,4,20180423,NaN
381,AN17051903980001,N,512,19510109,NaN,10.72,90,M,63.84953,0,...,NaN,PN102228158,0.3125\0.3125,440,512,OAx T1,6,4,20170520,NaN
417,AN16062001900025,N,512,19621212,NaN,1.636,12,F,63.849548,0,...,NaN,PN101934022,0.4883\0.4883,5.532,512,AX T1 FSPGR +c 1.75Ph,10,4,20160620,NaN
516,AN16061004660265,N,256,19680523,NaN,82.9,90,M,63.849511,0,...,NaN,PN101925204,0.9766\0.9766,6550,256,OAx DWI b=800,6,4,20160610,NaN
517,AN16061004660265,N,512,19680523,NaN,1.636,12,M,63.849511,0,...,NaN,PN101925204,0.4883\0.4883,5.528,512,AX T1 FSPGR +c 1.75Ph,7,5,20160610,NaN


In [141]:
HeadInform[(HeadInform.NiiPath.isnull())& ].index[0]

0

In [150]:
HeadInformUse=HeadInform[~(HeadInform.NiiPath.isnull())]
# HeadInformUse.loc[:,'FileName']=[i[0].split('/')[-1].split('_')[0] for i in HeadInformUse.NiiPath]

In [152]:
HeadInformUse.loc[:,'FolderName']=[i.split('/')[-1] for i in HeadInformUse.DicomPath]

/home/nye/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/nye/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [155]:
set(HeadInformUse.FolderName)

{'DCE', 'DWI', 'T2', 'T2WI'}

In [160]:
HeadInformUse.loc[:,'FileName']=[i[0].split('/')[-1].split('_')[0] for i in HeadInformUse.NiiPath]

/home/nye/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [163]:
HeadInformUse.loc[:,'FolderNameUse']=HeadInformUse.FolderName.map({'T2':'T2','DCE':'T1','DWI':'DWI','T2WI':'T2'})

In [161]:
HeadInformUse.FileName[0]

'T2'

In [172]:
len(SerNumberError),len(SerErrorDf)

(873, 873)

In [174]:
SerNameError=[i!=j for i,j in zip(HeadInformUse.FolderNameUse,HeadInformUse.FileName)]
SerNumberError=(HeadInformUse.NumberOfSlice<=15) | (HeadInformUse.NumberOfSlice>=44)
# NoNii=HeadInform.NiiPath.isnull()
AlignmentDf=HeadInformUse[~(SerNameError|SerNumberError)]

In [166]:
SerErrorDf=HeadInformUse[SerNameError]
SerErrorDf.to_excel('./SerErrorDf.xlsx')

In [181]:
check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh','.','.'])[0]

'1'

In [223]:
AllNeedCheck=HeadInformUse[(SerNameError|SerNumberError)]

In [225]:
NeedCheckFinial=pd.concat([HeadInform[(HeadInform.NiiPath.isnull())],HeadInformUse[(SerNameError)],HeadInformUse[(SerNumberError)]],
          keys=['CantTrans','SerNameError','NumberPerSerError'])

In [226]:
NeedCheckFinial.to_excel('./NeedCheckFinial.xlsx')

In [236]:
NeedCheckFinial.columns

Index([u'AccessionNumber_0008_0050', u'AngioFlag_0018_0025',
       u'Columns_0028_0011', u'DOB_0010_0030', u'DicomPath',
       u'EchoTime_0018_0081', u'FileName', u'FlipAngle_0018_1314',
       u'FolderName', u'FolderNameUse', u'Gender_0010_0040',
       u'ImagingFrequency_0018_0084', u'InversionTime_0018_0082',
       u'MRAcquisitionType_0018_0023', u'MagneticFieldStrength_0018_0087',
       u'Manufacturer_0008_0070', u'NiiPath', u'NubFiles', u'NumberOfSlice',
       u'PatientID_0010_0040', u'PixelSpacing_0028_0030',
       u'RepetitionTime_0018_0080', u'Rows_0028_0010',
       u'SeriesDescription_0008_103e', u'SeriesNumber_0020_0011',
       u'SliceThickness_0018_0050', u'StudyDate_0008_0020',
       u'TotalNumberDicom'],
      dtype='object')

In [227]:
Usable=['102075302','102390108','102396205','102426454']

In [238]:
UsableDf=NeedCheckFinial[NeedCheckFinial.PatientID_0010_0040.isin(['PN'+i+' ' for i in Usable])].drop_duplicates(['SeriesDescription_0008_103e','PatientID_0010_0040'])

In [246]:
UsableDf.reset_index().drop(['level_1','level_0'],axis=1,inplace=True)

In [247]:
AlignmentDf1=pd.concat([UsableDf,AlignmentDf],ignore_index=True).reset_index().drop('index',axis=1)

In [249]:
AlignmentDf1.shape

(850, 28)

In [253]:
from subprocess import call,check_output

def AlignmentFun(i):
    Nii=AlignmentDf1.NiiPath[i][0]
    Path='/'.join(Nii.split('/')[:-1])
    Raw='/home2/nye/SH_MH_W/AlignmentRaw/'+Nii.split('/')[-2]
    Use='/home2/nye/SH_MH_W/AlignmentUse/'+Nii.split('/')[-2]
    if 'T1' in Nii:

        PathUse=Path+'/T1*'
        call(['/home/nye/Documents/SH_MH_W/T1merge.sh',PathUse,Raw,Use])
    elif 'T2' in Nii:
        PathUse=Nii
        call(['mkdir','-p',Use])
        call(['cp',PathUse,Use+'/T2.nii.gz'])
    elif 'DWI' in Nii:
        PathUse=Path+'/DWI*'
        call(['/home/nye/Documents/SH_MH_W/ADC_generater.sh',PathUse,'800',Raw,Use])
        
    # check
#     print PathUse
    FinialF=Use+'/'+PathUse.split('/')[-1].split('_')[0]+'*'
    C=check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',PathUse,FinialF])
#     print PathUse,FinialF
    if C[0]=='1':
        return [True]
    else:
        return [False]

In [221]:
AlignmentFun(HeadInform[~HeadInform.NiiPath.isnull()].index[2])

[True]

In [254]:
from subprocess import call
from multiprocessing import Pool
a=list(Pool(processes=2).imap(AlignmentFun,AlignmentDf1.index))

# updated patient 20181010

In [35]:
RootNeed=[]
for root,folder,files in os.walk('/home2/nye/SH_MH_W/Updated/New/'):
    if files!=[] and any('dcm' in i for i in files):
        RootNeed.append(root)

In [41]:
FinialPathRoot='/home2/nye/SH_MH_W/NiiUse_updated'
# CantRun=[]
# import random

# for root,folder,files in os.walk('/home2/nye/SH_MH_W/RawDicom/thy1/'):
#     if files!=[] and any('dcm' in i for i in files):
        
def tempfun(root):
    N=[]
    Fr=[]
    CantRun={}
    FinialPathRootUse=os.path.join(FinialPathRoot,root.split('/')[-2])
    try:
        os.makedirs(FinialPathRootUse)
    except:
        pass
#         print  FinialPathRootUse
    try:
        N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                                  root=root,
                                  InformationNeed=InformationNeed)
    except:
        Files=os.listdir(root)
        CantRun={'Path':root,'Files':Files,'FileNub':len(list(filter(lambda x: 'dcm' in x,Files)))}

    return N,Fr,CantRun

from subprocess import call
# from multiprocessing import Pool
b=map(tempfun,RootNeed)

In [43]:
CantRun=[]
NeedCheck=[]
Result=[]
for x,y,z in b:
    if z!={}:
        CantRun.append(z)
    NeedCheck.extend(x)
    Result.extend(y)

In [45]:
CantRun

[]

In [47]:
import pandas as pd
HeadInform=pd.DataFrame({n:i for n,i in enumerate(Result)}).T



# HeadInform.head()

HeadInform.to_excel('./HeadInform_added_20181010.xlsx')

NeedCheckDf=pd.DataFrame({n:i for n,i in enumerate(NeedCheck)}).T

NeedCheckDf.to_excel('./NeedCheckDf_added_20181010.xlsx')

In [1]:
import pandas as pd
HeadInform=pd.read_excel('./HeadInform_added_20181010.xlsx')

In [48]:
NeedCheckDf

,Files,NubFiles,Path
0,"[PN101934022_20160620_MR_10_288_001.dcm, PN101...",288,/home2/nye/SH_MH_W/Updated/New/101934022/DCE
1,"[pn101815259_20160218_6_17_01.dcm, pn101815259...",51,/home2/nye/SH_MH_W/Updated/New/101815259/DWI


In [3]:
HeadInformUse=HeadInform[~(HeadInform.NiiPath.isnull())]
# HeadInformUse.loc[:,'FileName']=[i[0].split('/')[-1].split('_')[0] for i in HeadInformUse.NiiPath]

In [17]:
!ls /home2/nye/SH_MH_W/NiiUse/101040909

DWI_6_40_1.nii.gz  T1_7_320_2.nii.gz  T1_7_320_5.nii.gz  T1_7_320_8.nii.gz
DWI_6_40_2.nii.gz  T1_7_320_3.nii.gz  T1_7_320_6.nii.gz  T2_3_11_1.nii.gz
T1_7_320_1.nii.gz  T1_7_320_4.nii.gz  T1_7_320_7.nii.gz  T2_5_18_1.nii.gz


In [12]:
import re
re.findall('\'(.*?)\'',HeadInformUse.NiiPath[2])


[u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_1.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_2.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_3.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_4.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_5.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_6.nii.gz',
 u'/home2/nye/SH_MH_W/NiiUse_updated/101040909/T1_7_320_7.nii.gz']

In [19]:
from subprocess import call,check_output
import os
import re
def AlignmentFun(i):
    NiiList=re.findall('\'(.*?)\'',HeadInformUse.NiiPath[i])
    Nii=NiiList[0]
    Path='/'.join(Nii.split('/')[:-1])
    Raw='/home2/nye/SH_MH_W/AlignmentRaw_update/'+Nii.split('/')[-2]
    Use='/home2/nye/SH_MH_W/AlignmentUse_update/'+Nii.split('/')[-2]
    Files=[]
    try:
        Files=os.listdir(Use)
    except:
        pass
    if 'T1' in Nii:
        T1=filter(lambda x:re.search('T1',x)!=None,Files)
        if len(T1)==len(NiiList):
            pass
        else:
            PathUse=Path+'/T1*'
            call(['/home/nye/Documents/SH_MH_W/T1merge.sh',PathUse,Raw,Use])
    elif 'T2' in Nii:
        PathUse=Nii
        call(['mkdir','-p',Use])
        call(['cp',PathUse,Use+'/T2.nii.gz'])
    elif 'DWI' in Nii:
        DWI=filter(lambda x:re.search('DWI|ADC',x)!=None,Files)
        if len(DWI)==3:
            pass
        else:
            PathUse=Path+'/DWI*'
            call(['/home/nye/Documents/SH_MH_W/ADC_generater.sh',PathUse,'800',Raw,Use])
        
    # check
#     print PathUse
    FinialF=Use+'/'+PathUse.split('/')[-1].split('_')[0]+'*'
    C=check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',PathUse,FinialF])
#     print PathUse,FinialF
    if C[0]=='1':
        return [True]
    else:
        return [False]

# AlignmentFun(HeadInform[~HeadInform.NiiPath.isnull()].index[2])

from subprocess import call
from multiprocessing import Pool
a=list(Pool(processes=2).imap(AlignmentFun,HeadInformUse.index))

Process PoolWorker-1:
Process PoolWorker-2:
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
    self.run()
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/queues.py", line 374, in get
  File "/home/nye/anaconda2/lib/python2.7/multiprocessing/queues.py", line 376, in 

In [32]:
RootNeed=[]
for root,folder,files in os.walk('/home2/nye/SH_MH_W/Updated/update20181016/'):
    if files!=[] and any('dcm' in i for i in files):
        RootNeed.append(root)
FinialPathRoot='/home2/nye/SH_MH_W/Updated/update20181016_Nii/'
def tempfun(root):
    N=[]
    Fr=[]
    CantRun={}
    FinialPathRootUse=os.path.join(FinialPathRoot,root.split('/')[-2])
    try:
        os.makedirs(FinialPathRootUse)
    except:
        pass
#         print  FinialPathRootUse
#     try:
    N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                              root=root,
                              InformationNeed=InformationNeed)
#     except:
#         Files=os.listdir(root)
#         CantRun={'Path':root,'Files':Files,'FileNub':len(list(filter(lambda x: 'dcm' in x,Files)))}

    return N,Fr,CantRun

from subprocess import call
# from multiprocessing import Pool
b=map(tempfun,RootNeed)
CantRun=[]
NeedCheck=[]
Result=[]
for x,y,z in b:
    if z!={}:
        CantRun.append(z)
    NeedCheck.extend(x)
    Result.extend(y)

In [34]:
import pandas as pd
HeadInform=pd.DataFrame({n:i for n,i in enumerate(Result)}).T



# HeadInform.head()

HeadInform.to_excel('./HeadInform_added_20181016.xlsx')

NeedCheckDf=pd.DataFrame({n:i for n,i in enumerate(NeedCheck)}).T

NeedCheckDf.to_excel('./NeedCheckDf_added_20181016.xlsx')

In [37]:
HeadInformUse=HeadInform[~(HeadInform.NiiPath.isnull())]

In [41]:
from subprocess import call,check_output
import os
# import re
def AlignmentFun(i):
    NiiList=HeadInformUse.NiiPath[i]
    Nii=NiiList[0]
    Path='/'.join(Nii.split('/')[:-1])
    Raw='/home2/nye/SH_MH_W/Updated/update20181016_AlignmentRaw/'+Nii.split('/')[-2]
    Use='/home2/nye/SH_MH_W/Updated/update20181016_AlignmentUse/'+Nii.split('/')[-2]
    Files=[]
    try:
        Files=os.listdir(Use)
    except:
        pass
    if 'T1' in Nii:
        T1=filter(lambda x:re.search('T1',x)!=None,Files)
        if len(T1)==len(NiiList):
            pass
        else:
            PathUse=Path+'/T1*'
            call(['/home/nye/Documents/SH_MH_W/T1merge.sh',PathUse,Raw,Use])
    elif 'T2' in Nii:
        PathUse=Nii
        call(['mkdir','-p',Use])
        call(['cp',PathUse,Use+'/T2.nii.gz'])
    elif 'DWI' in Nii:
        DWI=filter(lambda x:re.search('DWI|ADC',x)!=None,Files)
        if len(DWI)==3:
            pass
        else:
            PathUse=Path+'/DWI*'
            call(['/home/nye/Documents/SH_MH_W/ADC_generater.sh',PathUse,'800',Raw,Use])
        
    # check
#     print PathUse
    FinialF=Use+'/'+PathUse.split('/')[-1].split('_')[0]+'*'
    C=check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',PathUse,FinialF])
#     print PathUse,FinialF
    if C[0]=='1':
        return [True]
    else:
        return [False]

# AlignmentFun(HeadInform[~HeadInform.NiiPath.isnull()].index[2])

from subprocess import call
from multiprocessing import Pool
a=list(Pool(processes=2).imap(AlignmentFun,HeadInformUse.index))

In [42]:
a

[[True], [True], [True]]

In [43]:
RootNeed=[]
for root,folder,files in os.walk('/home2/nye/SH_MH_W/Updated/20181018/'):
    if files!=[] and any('dcm' in i for i in files):
        RootNeed.append(root)
FinialPathRoot='/home2/nye/SH_MH_W/Updated/20181018_Nii/'
def tempfun(root):
    N=[]
    Fr=[]
    CantRun={}
    FinialPathRootUse=os.path.join(FinialPathRoot,root.split('/')[-2])
    try:
        os.makedirs(FinialPathRootUse)
    except:
        pass
#         print  FinialPathRootUse
#     try:
    N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                              root=root,
                              InformationNeed=InformationNeed)
#     except:
#         Files=os.listdir(root)
#         CantRun={'Path':root,'Files':Files,'FileNub':len(list(filter(lambda x: 'dcm' in x,Files)))}

    return N,Fr,CantRun

from subprocess import call
# from multiprocessing import Pool
b=map(tempfun,RootNeed)
CantRun=[]
NeedCheck=[]
Result=[]
for x,y,z in b:
    if z!={}:
        CantRun.append(z)
    NeedCheck.extend(x)
    Result.extend(y)

import pandas as pd
HeadInform=pd.DataFrame({n:i for n,i in enumerate(Result)}).T



# HeadInform.head()

HeadInform.to_excel('./HeadInform_added_20181018.xlsx')

NeedCheckDf=pd.DataFrame({n:i for n,i in enumerate(NeedCheck)}).T

NeedCheckDf.to_excel('./NeedCheckDf_added_20181018.xlsx')

HeadInformUse=HeadInform[~(HeadInform.NiiPath.isnull())]

from subprocess import call,check_output
import os
# import re
def AlignmentFun(i):
    NiiList=HeadInformUse.NiiPath[i]
    Nii=NiiList[0]
    Path='/'.join(Nii.split('/')[:-1])
    Raw='/home2/nye/SH_MH_W/Updated/20181018_AlignmentRaw/'+Nii.split('/')[-2]
    Use='/home2/nye/SH_MH_W/Updated/20181018_AlignmentUse/'+Nii.split('/')[-2]
    Files=[]
    try:
        Files=os.listdir(Use)
    except:
        pass
    if 'T1' in Nii:
        T1=filter(lambda x:re.search('T1',x)!=None,Files)
        if len(T1)==len(NiiList):
            pass
        else:
            PathUse=Path+'/T1*'
            call(['/home/nye/Documents/SH_MH_W/T1merge.sh',PathUse,Raw,Use])
    elif 'T2' in Nii:
        PathUse=Nii
        call(['mkdir','-p',Use])
        call(['cp',PathUse,Use+'/T2.nii.gz'])
    elif 'DWI' in Nii:
        DWI=filter(lambda x:re.search('DWI|ADC',x)!=None,Files)
        if len(DWI)==3:
            pass
        else:
            PathUse=Path+'/DWI*'
            call(['/home/nye/Documents/SH_MH_W/ADC_generater.sh',PathUse,'800',Raw,Use])
        
    # check
#     print PathUse
    FinialF=Use+'/'+PathUse.split('/')[-1].split('_')[0]+'*'
    C=check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',PathUse,FinialF])
#     print PathUse,FinialF
    if C[0]=='1':
        return [True]
    else:
        return [False]

# AlignmentFun(HeadInform[~HeadInform.NiiPath.isnull()].index[2])

from subprocess import call
# from multiprocessing import Pool
map(AlignmentFun,HeadInformUse.index)

[[True]]

In [11]:
# RootNeed=[]
# import os
# for root,folder,files in os.walk('/home2/nye/SH_MH_W/Updated/thyroid20181023/Dicom-supplement/'):
#     if files!=[] and any('dcm' in i for i in files):
#         RootNeed.append(root)
# FinialPathRoot='/home2/nye/SH_MH_W/Updated/thyroid20181023/Dicom-supplement_nii/'
# def tempfun(root):
#     N=[]
#     Fr=[]
#     CantRun={}
#     FinialPathRootUse=os.path.join(FinialPathRoot,root.split('/')[-2])
#     try:
#         os.makedirs(FinialPathRootUse)
#     except:
#         pass
# #         print  FinialPathRootUse
# #     try:
#     N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
#                               root=root,
#                               InformationNeed=InformationNeed)
# #     except:
# #         Files=os.listdir(root)
# #         CantRun={'Path':root,'Files':Files,'FileNub':len(list(filter(lambda x: 'dcm' in x,Files)))}

#     return N,Fr,CantRun

# from subprocess import call
# # from multiprocessing import Pool
# b=map(tempfun,RootNeed)
# CantRun=[]
# NeedCheck=[]
# Result=[]
# for x,y,z in b:
#     if z!={}:
#         CantRun.append(z)
#     NeedCheck.extend(x)
#     Result.extend(y)

# import pandas as pd
# HeadInform=pd.DataFrame({n:i for n,i in enumerate(Result)}).T



# # HeadInform.head()

# HeadInform.to_excel('./BaseInform/HeadInform_added_20181024.xlsx')

# NeedCheckDf=pd.DataFrame({n:i for n,i in enumerate(NeedCheck)}).T

# NeedCheckDf.to_excel('./BaseInform/NeedCheckDf_added_20181024.xlsx')

# HeadInformUse=HeadInform[~(HeadInform.NiiPath.isnull())]

from subprocess import call,check_output
import os
# import re
def AlignmentFun(i):
    NiiList=HeadInformUse.NiiPath[i]
    Nii=NiiList[0]
    Path='/'.join(Nii.split('/')[:-1])
    Raw='/home2/nye/SH_MH_W/Updated/thyroid20181023/Dicom-supplement_AlignmentRaw/'+Nii.split('/')[-2]
    Use='/home2/nye/SH_MH_W/Updated/thyroid20181023/Dicom-supplement_AlignmentUse/'+Nii.split('/')[-2]
    Files=[]
    try:
        Files=os.listdir(Use)
    except:
        pass
    if 'T1' in Nii:
        T1=filter(lambda x:re.search('T1',x)!=None,Files)
        if len(T1)==len(NiiList):
            pass
        else:
            PathUse=Path+'/T1*'
            call(['/home/nye/Documents/SH_MH_W/T1merge.sh',PathUse,Raw,Use])
    elif 'T2' in Nii:
        PathUse=Nii
        call(['mkdir','-p',Use])
        call(['cp',PathUse,Use+'/'+PathUse.split('/')[-1]])
    elif 'DWI' in Nii:
        DWI=filter(lambda x:re.search('DWI|ADC',x)!=None,Files)
        if len(DWI)==3:
            pass
        else:
            PathUse=Path+'/DWI*'
            call(['/home/nye/Documents/SH_MH_W/ADC_generater.sh',PathUse,'800',Raw,Use])
        
    # check
#     print PathUse
    FinialF=Use+'/'+PathUse.split('/')[-1].split('_')[0]+'*'
    C=check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',PathUse,FinialF])
#     print PathUse,FinialF
    if C[0]=='1':
        return [True]
    else:
        return [False]

# # AlignmentFun(HeadInform[~HeadInform.NiiPath.isnull()].index[2])

from subprocess import call
from multiprocessing import Pool
a=list(Pool(processes=2).imap(AlignmentFun,HeadInformUse.index))

In [202]:
aa='/home2/nye/SH_MH_W/NiiUse/101915985/DWI*'.split('/')[-1].split('_')[0]
x='/home2/nye/SH_MH_W/NiiUse/101915985/DWI*'

In [200]:
bb='/home2/nye/SH_MH_W/AlignmentUse/101915985'+'/'+aa

In [203]:
x

'/home2/nye/SH_MH_W/NiiUse/101915985/DWI*'

In [204]:
check_output(['/home/nye/Documents/SH_MH_W/CheckOutput.sh',x,bb])

'0\n'

In [42]:
RootNeed_updated=[]
for root,folder,files in os.walk('/home2/nye/SH_MH_W/Updated'):
    if files!=[] and any('dcm' in i for i in files):
        RootNeed.append(root)

In [43]:
for i in RootNeed_updated:
    x,y,z=tempfun(i)
    if z!={}:
        CantRun.append(z)
    NeedCheck.extend(x)
    Result.extend(y)

In [44]:
import pandas as pd
HeadInform=pd.DataFrame({n:x for n,x in enumerate(Result)}).T

In [33]:
FinialPathRoot='.\\NiiUse'
NeedCheck1=[]
for root in set(NeedCheck):
    FinialPathRootUse=os.path.join(FinialPathRoot,root.split('\\')[-2])
    try:
        N,Fr=Regular_Image2Ser(FinialPathRoot=FinialPathRootUse,
                              root=root,
                              InformationNeed=InformationNeed)
    except:
        try:
            N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                                  root=root,
                                  InformationNeed=InformationNeed)
        except:
            try:
                Number=random.randrange(10)
                NewPath= os.path.join(*root.split('\\')[:-1]+['RandomChangedName%d' %Number])
                os.rename(root,NewPath)
                N,Fr=Special_Image2Ser(FinialPathRoot=FinialPathRootUse,
                                      root=NewPath,
                                      InformationNeed=InformationNeed)
            except:
                print(NewPath)
                CantRun.append(NewPath)
    NeedCheck1.extend(N)
    Result.extend(Fr)

In [34]:
NeedCheck1

['D:\\SH_MH_W\\260-293\\1\\100529933\\DWI',
 'D:\\SH_MH_W\\260-293\\1\\100539031\\+C',
 'D:\\SH_MH_W\\120-154\\102272026\\T2WI',
 'D:\\SH_MH_W\\225-259\\102514321\\+C',
 'D:\\SH_MH_W\\260-293\\1\\102611664\\+C',
 'D:\\SH_MH_W\\260-293\\1\\102574360\\+C',
 'D:\\SH_MH_W\\260-293\\1\\102574360\\+C',
 'D:\\SH_MH_W\\260-293\\1\\102566947\\DWI',
 'D:\\SH_MH_W\\225-259\\102524505\\+C']

In [35]:
import pandas as pd
HeadInform=pd.DataFrame({n:x for n,x in enumerate(Result)}).T

In [41]:
HeadInform.PatientID_0010_0040[0]

'PN100190321 '

In [43]:
HeadInformUse=HeadInform[~HeadInform.DicomPath.isna()]

In [44]:
HeadInformUse.to_excel('./HeadInform.xlsx')

NameError: name 'ln' is not defined